# Modelo

In [1]:
nomemodelo = 'Light GBM'

## Caminhos

In [2]:
import os

#Folder Inicial
path = os.getcwd()

#Subpastas
pathin = path + '\\Entrada\\'
pathfixo = path + '\\Fixo\\'
pathout = path + '\\Saida\\'
pathparcial = path + '\\Parcial\\'
pathaux = path + '\\Auxiliar\\'

## Pacotes

In [3]:
import dill
import pickle
import pandas as pd
pd.set_option('max_colwidth', 3000)

import numpy as np

from time import gmtime, strftime

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
import lightgbm as lgb

from sklearn.externals import joblib

from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
from sklearn.utils import parallel_backend

In [4]:
import sklearn
print(sklearn.__version__)

0.20.3


## Lendo a Base

In [5]:
file = 'Treino'
treino = pd.read_pickle(pathparcial + 'Arquivo1 ' + file + '.pkl')

file = 'Teste'
teste = pd.read_pickle(pathparcial + 'Arquivo1 ' + file + '.pkl')

print(treino.shape)
print(teste.shape)

(10877, 11)
(3626, 11)


In [6]:
#Excluindo variaveis que nao serao usadas no modelo
y = 'respostafinal'
Xtr = treino[['texto', 'length', 'words', 'avg_word_length', 'min_word_length', 'max_word_length']]
colunas = list(Xtr)

with open(pathaux + 'Variaveis Modelo ' + nomemodelo + '.pickle', 'wb') as f:
    dill.dump((colunas, y), f)

Xte = teste[colunas]

print(Xtr.shape)
print(Xte.shape)

(10877, 6)
(3626, 6)


## Modelos

In [7]:
nomemodelo = 'Light GBM'

seed = 123

parameters = {
    'transformer__tfidf__max_features': [1000],
    #'transformer__tfidf__ngram_range': [(1, 1), (1, 2)],
    'transformer__tfidf__ngram_range': [(1, 1)],
    'transformer__tfidf__max_df': [.6],
    'transformer__tfidf__min_df': [20],
    'clf__max_depth': [4, 5, -1],
    'clf__bagging_fraction': [.7],
    'clf__min_data_leaf': [20],
    'clf__feature_fraction': [.7,],
    'clf__boosting_type': ['gbdt'],
    'clf__num_boost_round': [300],
    'clf__learning_rate': [.01],
    'clf__min_child_smaples': [20],
    'clf__class_weight': ['balanced', None],
    'clf__random_state': [seed]
}

pipeline = Pipeline([
    ('transformer', ColumnTransformer([('tfidf', TfidfVectorizer(analyzer = 'word'), 'texto')], remainder = 'passthrough')),
    ('clf', lgb.LGBMClassifier() )
])

cv = RepeatedStratifiedKFold(n_splits = 3, n_repeats = 1, random_state = seed)
gs = GridSearchCV(pipeline, parameters, cv = cv, scoring = 'f1_macro', n_jobs = 3, verbose = 1, refit = True)

In [8]:
#Rodando GridSearch
print(strftime('%Y-%m-%d %H:%M:%S', gmtime()))

with parallel_backend('multiprocessing'):
    gs.fit(Xtr, treino[y])

joblib.dump(gs, pathaux + 'Modelo ' + nomemodelo + '.pkl')

print(strftime('%Y-%m-%d %H:%M:%S', gmtime()))

2019-05-12 01:31:49
Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=3)]: Using backend MultiprocessingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  18 out of  18 | elapsed:   25.0s finished
C:\Users\laran\Anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


2019-05-12 01:33:08


In [9]:
gs.best_score_

0.4296979721617831

In [10]:
gs.best_params_

{'clf__bagging_fraction': 0.7,
 'clf__boosting_type': 'gbdt',
 'clf__class_weight': 'balanced',
 'clf__feature_fraction': 0.7,
 'clf__learning_rate': 0.01,
 'clf__max_depth': 5,
 'clf__min_child_smaples': 20,
 'clf__min_data_leaf': 20,
 'clf__num_boost_round': 300,
 'clf__random_state': 123,
 'transformer__tfidf__max_df': 0.6,
 'transformer__tfidf__max_features': 1000,
 'transformer__tfidf__min_df': 20,
 'transformer__tfidf__ngram_range': (1, 1)}

In [11]:
cvresults = pd.DataFrame(gs.cv_results_)[[
 'mean_test_score',
 'mean_train_score',
 'param_clf__max_depth',
 'param_clf__bagging_fraction',
 'param_clf__min_data_leaf',
 'param_clf__feature_fraction',
 'param_clf__boosting_type',
 'param_clf__num_boost_round',
 'param_clf__learning_rate',
 'param_clf__min_child_smaples',
 'param_clf__class_weight',
 'param_transformer__tfidf__max_df',
 'param_transformer__tfidf__max_features',
 'param_transformer__tfidf__min_df',
 'param_transformer__tfidf__ngram_range',
 'std_test_score',
 'std_train_score']]
ha = list(cvresults)
ha = [w.replace('param_clf__', '') for w in ha]
ha = [w.replace('param_transformer__', '') for w in ha]
cvresults.columns = ha
cvresults.to_excel(pathout + 'Modelo ' + nomemodelo + ' Resultados GridSearch.xlsx', encoding = 'latin1', index = False)

C:\Users\laran\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\laran\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\laran\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\laran\Anaconda3\lib\site-packages\sklearn\util

In [12]:
cvresults

,mean_test_score,mean_train_score,max_depth,bagging_fraction,min_data_leaf,feature_fraction,boosting_type,num_boost_round,learning_rate,min_child_smaples,class_weight,tfidf__max_df,tfidf__max_features,tfidf__min_df,tfidf__ngram_range,std_test_score,std_train_score
0,0.422259,0.449601,4,0.7,20,0.7,gbdt,300,0.01,20,balanced,0.6,1000,20,"(1, 1)",0.014672,0.001832
1,0.429698,0.455497,5,0.7,20,0.7,gbdt,300,0.01,20,balanced,0.6,1000,20,"(1, 1)",0.013734,0.001277
2,0.420964,0.480571,-1,0.7,20,0.7,gbdt,300,0.01,20,balanced,0.6,1000,20,"(1, 1)",0.011622,0.005924
3,0.353935,0.365801,4,0.7,20,0.7,gbdt,300,0.01,20,None,0.6,1000,20,"(1, 1)",0.007432,0.012052
4,0.362038,0.377660,5,0.7,20,0.7,gbdt,300,0.01,20,None,0.6,1000,20,"(1, 1)",0.007194,0.010598
5,0.381858,0.420923,-1,0.7,20,0.7,gbdt,300,0.01,20,None,0.6,1000,20,"(1, 1)",0.015490,0.007362


## Variaveis Importantes

In [13]:
def feature_names_tfidf(X, column_tfidf, **params_tfidf):
    tfidf = TfidfVectorizer(**params_tfidf)
    tfidf.fit(X[column_tfidf])
    tfidf_feature_names = {'tdidf__' + x for x in tfidf.get_feature_names()}
    feature_names = list(tfidf_feature_names) + list(X.columns.drop(column_tfidf))
    return feature_names

In [14]:
featurenames = feature_names_tfidf(X = Xtr, column_tfidf = 'texto',
    max_df = list(gs.best_params_.values())[list(gs.best_params_).index('transformer__tfidf__max_df')],
    min_df = list(gs.best_params_.values())[list(gs.best_params_).index('transformer__tfidf__min_df')],
    max_features = list(gs.best_params_.values())[list(gs.best_params_).index('transformer__tfidf__max_features')],
    ngram_range = list(gs.best_params_.values())[list(gs.best_params_).index('transformer__tfidf__ngram_range')]
)

In [15]:
def Feature_Importance(mod, nomemodelo, X, y, featurenames):
    many = ['Linear SVM', 'Logistica', 'Naive Bayes']

    k = mod.best_estimator_.named_steps['clf']

    if nomemodelo in many:
        #Logistica, SVM, Naive Bayes
        Features = pd.DataFrame(k.coef_.tolist())
        Features.columns = featurenames
        if len(y.unique()) > 2:
            Features.index = sorted(y.unique())
    else:
        #Arvores
        Features = pd.DataFrame({'features': featurenames, 'value': k.feature_importances_.tolist()})
        
    Features.to_excel(pathout + 'Features ' + nomemodelo + '.xlsx', encoding = 'latin1', index = True)
    
Feature_Importance(mod = gs, nomemodelo = nomemodelo, X = Xtr, y = treino[y], featurenames = featurenames)

## Preditos

In [16]:
predtr = gs.best_estimator_.predict(Xtr)
predte = gs.best_estimator_.predict(Xte)

In [17]:
name = 'Treino'
fim = treino[['ido', 'id', y]]
fim['pred'] = predtr

fim['Acertou'] = np.where(fim[y] == fim['pred'], 1, 0)
fim.to_excel(pathout + 'Modelo ' + nomemodelo + ' Pred ' + name + '.xlsx', encoding = 'latin1', index = False)

del fim

name = 'Teste'
fim = teste[['ido', 'id', y]]
fim['pred'] = predte

fim['Acertou'] = np.where(fim[y] == fim['pred'], 1, 0)
fim.to_excel(pathout + 'Modelo ' + nomemodelo + ' Pred ' + name + '.xlsx', encoding = 'latin1', index = False)

C:\Users\laran\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\laran\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\laran\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 